# Парсинг html файлов

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os

from sklearn.metrics import jaccard_score
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

📌 **Комментарий к коду ниже**:

**Открывает и читает HTML-файл:**

Принимает путь к файлу (file_path) в качестве аргумента.

Открывает файл в режиме чтения ('r') с кодировкой UTF-8, чтобы корректно обрабатывать русские символы.

Считывает все содержимое файла в переменную html_content.

**Создает объект BeautifulSoup:**

Использует BeautifulSoup для парсинга HTML-кода из html_content.

'html.parser' указывает, что используется встроенный HTML-парсер.

**Извлекает механизм действия:**

Ищет раздел с идентификатором mechanism-of-action (soup.find(id="mechanism-of-action")).

Если раздел найден, ищет следующий за ним тег <p> (параграф) и извлекает текст из него, убирая пробелы в начале и конце (.text.strip()). Это предполагает, что описание механизма действия находится непосредственно под заголовком раздела.

Если раздел не найден, присваивает переменной mechanism значение 'No mechanism found'.


**Извлекает мишени (Targets):**

Инициализирует пустой список targets.

Если раздел mechanism-of-action найден, пытается найти таблицу, следующую за этим разделом (targets_table = mechanism_section.find_next('table')).

Если таблица найдена, ищет все строки (<tr>) в таблице.

Проходит по каждой строке, начиная со второй (чтобы пропустить строку заголовков) (for row in rows[1:]:).

В каждой строке ищет все ячейки (<td>).

Проверяет, что в строке есть как минимум три ячейки.

Извлекает название мишени (target), действие (action) и организм (organism) из соответствующих ячеек.

Форматирует информацию о мишени в строку f"{target_name} ({action}, {organism})".

Добавляет строку в список targets.

После обработки всех строк, объединяет все элементы списка targets в одну строку, разделяя их символом ; . Если список targets пуст, присваивает переменной targets значение 'No targets found'.

**Извлекает показания (Indication):**

Ищет раздел с идентификатором indication (soup.find(id="indication")).

Если раздел найден, ищет следующий за ним тег <p> (параграф) и извлекает текст из него, убирая пробелы в начале и конце (.text.strip()). Это предполагает, что показания находятся непосредственно под заголовком раздела.

Если раздел не найден, присваивает переменной indication значение 'No indication found'.

**Собирает данные в словарь:**

Создает словарь drug_data, содержащий извлеченную информацию: механизм действия, мишени и показания.

**Возвращает словарь:**

Функция возвращает словарь drug_data с извлеченной информацией.

**Кратко:** Функция парсит HTML-файл, ищет определенные разделы (механизм действия, мишени, показания) и извлекает информацию из этих разделов, используя BeautifulSoup. Результатом является словарь, содержащий извлеченные данные. Функция предполагает определенную структуру HTML-файла.

In [12]:
def parse_drug_info(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    mechanism_section = soup.find(id="mechanism-of-action")
    mechanism = mechanism_section.find_next('p').text.strip() if mechanism_section else 'No mechanism found'

    targets = []
    if mechanism_section:
        targets_table = mechanism_section.find_next('table')
        if targets_table:
            rows = targets_table.find_all('tr')
            for row in rows[1:]:  # Пропустить заголовок
                cells = row.find_all('td')
                if len(cells) >= 3:  # Проверяем, что есть хотя бы 3 ячейки
                    target_name = cells[0].text.strip()
                    action = cells[1].text.strip()
                    organism = cells[2].text.strip()
                    targets.append(f"{target_name} ({action}, {organism})")
    targets = "; ".join(targets) if targets else 'No targets found'

    indication_section = soup.find(id="indication")
    indication = indication_section.find_next('p').text.strip() if indication_section else 'No indication found'

    drug_data = {
        'Mechanism of Action': mechanism,
        'Targets': targets,
        'Indication': indication
    }

    return drug_data

📌 **Комментарий к коду ниже**:

**Принимает путь к директории:**

Обрабатывает HTML-файлы из указанной директории.

**Цикл по файлам:**

Перебирает только файлы с расширением .html.

**Обработка файла:**

Вызывает parse_drug_info для извлечения данных.

Добавляет код ATC из имени файла.

Обрабатывает ошибки при обработке.

**Создание DataFrame:**

Объединяет все данные в DataFrame.

**Сохранение в CSV:**

Сохраняет DataFrame в файл drug_data.csv.

In [13]:
import zipfile

zip_path = "drug_pages_solo.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(".")  # Распаковываем в текущую директорию

print("ZIP успешно распакован!")


ZIP успешно распакован!


In [14]:
import glob

html_files = glob.glob("drug_pages_solo/*.html")

if html_files:
    print(f"Найдено {len(html_files)} HTML-файлов.")
else:
    print("Файлы все еще не найдены. Проверь ZIP-архив!")


Найдено 134 HTML-файлов.


In [15]:
def process_multiple_files(directory):
    all_data = []

    for file_name in os.listdir(directory):
        if file_name.endswith('.html'):
            file_path = os.path.join(directory, file_name)
            try:
                drug_data = parse_drug_info(file_path)
                drug_data['ATC'] = os.path.splitext(file_name)[0]
                all_data.append(drug_data)
            except Exception as e:
                print(f"Error processing file {file_name}: {e}")

    df = pd.DataFrame(all_data)
    return df


directory_path = './drug_pages_solo'
df = process_multiple_files(directory_path)
df.to_csv('./drug_data.csv', index=False)

print("Data saved to drug_data.csv")

Data saved to drug_data.csv


In [16]:
df.head(3)

,Mechanism of Action,Targets,Indication,ATC
0,Not Available,"ADipeptidyl peptidase 4 (inhibitor, Humans)",Not Available,A10BH07
1,Nateglinide activity is dependent on the prese...,AATP-binding cassette sub-family C member 8 (i...,For the treatment of non-insulin dependent-dia...,A10BX03
2,Not Available,"ASodium/glucose cotransporter 2 (modulator, Hu...",Not Available,A10BK05


# Объединение файлов и результатов парсинга, создание фичей

## Объединение

In [17]:
existing_csv_path = 'ATC_categories_diabetis.csv'
existing_df = pd.read_csv(existing_csv_path, index_col=0)

In [45]:
existing_df

,ATC,ATC_info,ATC_hierarchy,used_in_diabetes,is_terminal_category,number_of_children
54,A12AA10,Calcium glucoheptonate,"['A12AA10', 'A12AA', 'A12A', 'A12', 'A']",True,1,1
125,A10XX01,Teplizumab,"['A10XX01', 'A10XX', 'A10X', 'A10', 'A']",True,1,1
127,A10XA01,Tolrestat,"['A10XA01', 'A10XA', 'A10X', 'A10', 'A']",True,1,1
130,A10BK08,Bexagliflozin,"['A10BK08', 'A10BK', 'A10B', 'A10', 'A']",True,1,1
131,A10BK07,Luseogliflozin,"['A10BK07', 'A10BK', 'A10B', 'A10', 'A']",True,1,1
...,...,...,...,...,...,...
5894,R01AA10,Metizoline,"['R01AA10', 'R01AA', 'R01A', 'R01', 'R']",True,1,1
6036,V09IA10,Technetium (99mtc)trofolastat chloride,"['V09IA10', 'V09IA', 'V09I', 'V09', 'V']",True,1,1
6163,V08CA10,Gadoxetic acid,"['V08CA10', 'V08CA', 'V08C', 'V08', 'V']",True,1,1
6208,V08AA10,Diodone,"['V08AA10', 'V08AA', 'V08A', 'V08', 'V']",True,1,1


In [46]:
df

,Mechanism of Action,Targets,Indication,ATC
0,Not Available,"ADipeptidyl peptidase 4 (inhibitor, Humans)",Not Available,A10BH07
1,Nateglinide activity is dependent on the prese...,AATP-binding cassette sub-family C member 8 (i...,For the treatment of non-insulin dependent-dia...,A10BX03
2,Not Available,"ASodium/glucose cotransporter 2 (modulator, Hu...",Not Available,A10BK05
3,No mechanism found,No targets found,No indication found,V09IA10
4,Not Available,No targets found,Not Available,A10BX01
...,...,...,...,...
129,"In order to replicate, bacteria require a spec...","A23S ribosomal RNA (inhibitor, Enteric bacteri...",Azithromycin should be used only to treat or p...,J01FA10
130,"In contrast to sulfonylureas, miglitol does no...","ALysosomal alpha-glucosidase (antagonist, Huma...",For use as an adjunct to diet to improve glyce...,A10BF02
131,Not Available,No targets found,Not Available,N05CA10
132,Dapagliflozin inhibits the sodium-glucose cotr...,ASodium/glucose cotransporter 2 (antagonistinh...,Dapagliflozin is indicated as an adjunct treat...,A10BK01


In [34]:
merged_df = pd.merge(existing_df, df, on='ATC', how='outer')

In [35]:
merged_df['ATC Code'] = merged_df['ATC'].str[:-2]

In [47]:
merged_df

,ATC,ATC_info,ATC_hierarchy,used_in_diabetes,is_terminal_category,number_of_children,Mechanism of Action,Targets,Indication,ATC Code
0,A02AA10,Combinations,"['A02AA10', 'A02AA', 'A02A', 'A02', 'A']",True,1,1,No mechanism found,No targets found,No indication found,A02AA
1,A03FA10,Acotiamide,"['A03FA10', 'A03FA', 'A03F', 'A03', 'A']",True,1,1,Not Available,No targets found,Not Available,A03FA
2,A05BA10,Phospholipids,"['A05BA10', 'A05BA', 'A05B', 'A05', 'A']",True,1,1,Omega-3 fatty acids mediate anti-inflammatory ...,APeroxisome proliferator-activated receptor ga...,Provided as daily supplements. Aa preparation ...,A05BA
3,A07AA10,Colistin,"['A07AA10', 'A07AA', 'A07A', 'A07', 'A']",True,1,1,Colistin is a surface active agent which penet...,"ADihydropteroate synthase (binder, Escherichia...",For the treatment of acute or chronic infectio...,A07AA
4,A08AA10,Sibutramine,"['A08AA10', 'A08AA', 'A08A', 'A08', 'A']",True,1,1,Sibutramine produces its therapeutic effects b...,ASodium-dependent dopamine transporter (inhibi...,For the treatment of obesity.,A08AA
...,...,...,...,...,...,...,...,...,...,...
168,R06AA10,Trimethobenzamide,"['R06AA10', 'R06AA', 'R06A', 'R06', 'R']",True,1,1,The mechanism of action of trimethobenzamide a...,"AD(2) dopamine receptor (inhibitor, Humans)",For the treatment of postoperative nausea and ...,R06AA
169,V01AA10,Flowers,"['V01AA10', 'V01AA', 'V01A', 'V01', 'V']",True,1,1,No mechanism found,No targets found,No indication found,V01AA
170,V08AA10,Diodone,"['V08AA10', 'V08AA', 'V08A', 'V08', 'V']",True,1,1,Not Available,No targets found,Not Available,V08AA
171,V08CA10,Gadoxetic acid,"['V08CA10', 'V08CA', 'V08C', 'V08', 'V']",True,1,1,Gadoxetate disodium is a paramagnetic compound...,No targets found,Gadoxetate is indicated for intravenous use in...,V08CA


In [48]:
all_drug_info_df=pd.read_csv("all_drug_info.csv")
all_drug_info_df['ATC Code'] = all_drug_info_df['ATC Code'].str.split('(').str[0].str.strip()

In [49]:
all_drug_info_df

,Drug Name,RxCUI,ATC Code,Ingredients,Clinical Dose Groups,Dose Form Groups,Branded Dose Groups
0,Calcium glucoheptonate,47618,A12AA,calcium gluceptate,NaN,NaN,NaN
1,Teplizumab,2621880,A10XX,teplizumab,teplizumab Injectable Product,Injectable Product,Tzield Injectable Product
2,Tolrestat,38386,A10XA,tolrestat,NaN,NaN,NaN
3,Bexagliflozin,2627044,A10BK,bexagliflozin,"bexagliflozin Oral Product, bexagliflozin Pill","Oral Product, Pill","Bexacat Oral Product, Bexacat Pill, Brenzavvy ..."
4,Sotagliflozin,2638675,A10BK,sotagliflozin,"sotagliflozin Oral Product, sotagliflozin Pill","Oral Product, Pill","Inpefa Oral Product, Inpefa Pill"
...,...,...,...,...,...,...,...
107,Cyclobarbital,2978,N05CA,cyclobarbital,NaN,NaN,NaN
108,Ketazolam,28181,N05BA,ketazolam,NaN,NaN,NaN
109,Benorilate,1372,N02BA,benorilate,"benorilate Oral Liquid Product, benorilate Ora...","Oral Product, Pill, Oral Liquid Product",NaN
110,Papaveretum,253192,N02AA,papaveretum,NaN,NaN,NaN


In [55]:
# Объединяем таблицы по 'ATC' с внешним соединением
final_df = pd.merge(merged_df, all_drug_info_df, on='ATC Code', how='inner')

# Сохраняем результат
final_df.to_csv("final_df.csv", index=False)

In [56]:
final_df

,ATC,ATC_info,ATC_hierarchy,used_in_diabetes,is_terminal_category,number_of_children,Mechanism of Action,Targets,Indication,ATC Code,Drug Name,RxCUI,Ingredients,Clinical Dose Groups,Dose Form Groups,Branded Dose Groups
0,A07AA10,Colistin,"['A07AA10', 'A07AA', 'A07A', 'A07', 'A']",True,1,1,Colistin is a surface active agent which penet...,"ADihydropteroate synthase (binder, Escherichia...",For the treatment of acute or chronic infectio...,A07AA,Colistin,2709,colistin,colistin / hydrocortisone / neomycin / thonzon...,"Injectable Product, Oral Product, Pill, Otic P...","Coly Mycin M Injectable Product, Cortisporin-T..."
1,A08AA10,Sibutramine,"['A08AA10', 'A08AA', 'A08A', 'A08', 'A']",True,1,1,Sibutramine produces its therapeutic effects b...,ASodium-dependent dopamine transporter (inhibi...,For the treatment of obesity.,A08AA,Sibutramine,1294535,sibutramine,"sibutramine Oral Product, sibutramine Pill","Oral Product, Pill",NaN
2,A10AB01,Insulin (human),"['A10AB01', 'A10AB', 'A10A', 'A10', 'A']",True,1,1,The primary activity of insulin is the regulat...,"AInsulin receptor (agonist, Humans); UInsulin-...",Human insulin is indicated to improve glycemic...,A10AB,Insulin (human),253182,"insulin, regular, human","insulin, regular, human Injectable Product, in...","Inhalant Product, Injectable Product","Humulin Injectable Product, Humulin R Injectab..."
3,A10AB01,Insulin (human),"['A10AB01', 'A10AB', 'A10A', 'A10', 'A']",True,1,1,The primary activity of insulin is the regulat...,"AInsulin receptor (agonist, Humans); UInsulin-...",Human insulin is indicated to improve glycemic...,A10AB,Insulin (pork),221109,"insulin, regular, pork","insulin, regular, pork Injectable Product, ins...",Injectable Product,NaN
4,A10AB01,Insulin (human),"['A10AB01', 'A10AB', 'A10A', 'A10', 'A']",True,1,1,The primary activity of insulin is the regulat...,"AInsulin receptor (agonist, Humans); UInsulin-...",Human insulin is indicated to improve glycemic...,A10AB,Insulin (human),253182,"insulin, regular, human","insulin, regular, human Injectable Product, in...","Inhalant Product, Injectable Product","Humulin Injectable Product, Humulin R Injectab..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882,N05BA10,Ketazolam,"['N05BA10', 'N05BA', 'N05B', 'N05', 'N']",True,1,1,Benzodiazepines share a similar chemical struc...,AGABA(A) Receptor (positive allosteric modulat...,Ketazolam could be used for the treatment of a...,N05BA,Ketazolam,28181,ketazolam,NaN,NaN,NaN
883,N05CA10,Cyclobarbital,"['N05CA10', 'N05CA', 'N05C', 'N05', 'N']",True,1,1,Not Available,No targets found,Not Available,N05CA,Cyclobarbital,2978,cyclobarbital,NaN,NaN,NaN
884,N06AA10,Nortriptyline,"['N06AA10', 'N06AA', 'N06A', 'N06', 'N']",True,1,1,Though prescribing information does not identi...,ASodium-dependent noradrenaline transporter (i...,Nortriptyline is indicated for the relief of t...,N06AA,Nortriptyline,203130,nortriptyline,NaN,NaN,NaN
885,N06BA10,Fenetylline,"['N06BA10', 'N06BA', 'N06B', 'N06', 'N']",True,1,1,Not Available,No targets found,Not Available,N06BA,Fenetylline,24840,fenethylline,NaN,NaN,NaN


## Создание фичей

In [57]:
data = pd.read_csv('final_df.csv', index_col = 0)

In [58]:
# Фича 1: Вытащить все слова по отдельности из текста (как идея проанализировать самые часто встречающиеся слова в описании исключив stopwords)
# или например для каких либо категорий посмотреть какие слова больше подходят
data['mechanism_keywords'] = data['Mechanism of Action'].str.findall(r'\b\w+\b').apply(lambda x: list(set(x)) if isinstance(x, list) else [])

In [63]:
# Фича 2.1: Подсчет количества целей
data['num_targets'] = data['Targets'].apply(lambda x: len(x.split(',')) if isinstance(x, str) else 0)
# Фича 2.2: Подсчет количества DFG
data['num_dfg'] = data['Dose Form Groups'].apply(lambda x: len(x.split(',')) if isinstance(x, str) else 0)
# Фича 2.3: Подсчет количества ингридиентов
data['num_ingridients'] = data['Ingredients'].apply(lambda x: len(x.split(',')) if isinstance(x, str) else 0)
# Фича 2.4: Подсчет количества слов в механизме действия чтобы прикинуть его сложность
data['mechanism_complexity'] = data['Mechanism of Action'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)

In [64]:
# Фича 4: Категоризация индикации в зависимости от слов.
def categorize_indication(indication):
    if not isinstance(indication, str):
        return 'unknown'
    if 'chronic' in indication.lower():
        return 'chronic'
    if 'acute' in indication.lower():
        return 'acute'
    if 'obesity' in indication.lower():
        return 'obesity'
    return 'other'

data['indication_category'] = data['Indication'].apply(categorize_indication)

In [65]:
# Фича 5: Просто отобразить есть ли пропущенные значения в этих столбцах.
data['missing_mechanism'] = data['Mechanism of Action'].str.contains('No mechanism found|Not Available', na=False)
data['missing_targets'] = data['Targets'].str.contains('No targets found|Not Available', na=False)
data['missing_indication'] = data['Indication'].str.contains('No indication found|Not Available', na=False)

In [66]:
data.head(5)

,ATC_info,ATC_hierarchy,used_in_diabetes,is_terminal_category,number_of_children,Mechanism of Action,Targets,Indication,ATC Code,Drug Name,...,Branded Dose Groups,mechanism_keywords,num_targets,mechanism_complexity,indication_category,num_dfg,num_ingridients,missing_mechanism,missing_targets,missing_indication
ATC,,,,,,,,,,,,,,,,,,,,,
A07AA10,Colistin,"['A07AA10', 'A07AA', 'A07A', 'A07', 'A']",True,1,1,Colistin is a surface active agent which penet...,"ADihydropteroate synthase (binder, Escherichia...",For the treatment of acute or chronic infectio...,A07AA,Colistin,...,"Coly Mycin M Injectable Product, Cortisporin-T...","[interacts, that, its, Colistin, effect, disru...",3,54,chronic,4,1,False,False,False
A08AA10,Sibutramine,"['A08AA10', 'A08AA', 'A08A', 'A08', 'A']",True,1,1,Sibutramine produces its therapeutic effects b...,ASodium-dependent dopamine transporter (inhibi...,For the treatment of obesity.,A08AA,Sibutramine,...,NaN,"[through, been, energy, that, promotes, its, s...",4,93,obesity,2,1,False,False,False
A10AB01,Insulin (human),"['A10AB01', 'A10AB', 'A10A', 'A10', 'A']",True,1,1,The primary activity of insulin is the regulat...,"AInsulin receptor (agonist, Humans); UInsulin-...",Human insulin is indicated to improve glycemic...,A10AB,Insulin (human),...,"Humulin Injectable Product, Humulin R Injectab...","[bound, inhibits, regulation, C, 1, binds, mus...",7,160,other,2,3,False,False,False
A10AB01,Insulin (human),"['A10AB01', 'A10AB', 'A10A', 'A10', 'A']",True,1,1,The primary activity of insulin is the regulat...,"AInsulin receptor (agonist, Humans); UInsulin-...",Human insulin is indicated to improve glycemic...,A10AB,Insulin (pork),...,NaN,"[bound, inhibits, regulation, C, 1, binds, mus...",7,160,other,1,3,False,False,False
A10AB01,Insulin (human),"['A10AB01', 'A10AB', 'A10A', 'A10', 'A']",True,1,1,The primary activity of insulin is the regulat...,"AInsulin receptor (agonist, Humans); UInsulin-...",Human insulin is indicated to improve glycemic...,A10AB,Insulin (human),...,"Humulin Injectable Product, Humulin R Injectab...","[bound, inhibits, regulation, C, 1, binds, mus...",7,160,other,2,3,False,False,False


In [67]:
data.to_csv('data_for_eda.csv')

Ниже идет рассчет двух фичей: <br>
- avg_target_overlap
- avg_mechanism_similarity

`avg_target_overlap` отражает среднее количество пересечений целей воздействия текущего лекарства с целями воздействия других лекарств в наборе данных. Он измеряет, насколько схожи цели (рецепторы, ферменты, белки и т. д.), на которые влияет данное лекарство, с целями других лекарств.
`avg_mechanism_similarity` вычисляет среднее значение коэффициента Жаккара между текстом механизма действия текущего лекарства и текстами механизмов действия всех других лекарств в наборе данных.


Принцип работы столбца avg_target_overlap

Сравнение целей:
Для текущего лекарства и каждого другого лекарства вычисляется пересечение множеств целей.<br>
Лекарство имеет большое количество общих целей с другими.<br>
Это может указывать на принадлежность препарата к широко используемому классу лекарств.<br>

Среднее значение (0.5 <= avg_target_overlap <= 1):<br>
Лекарство пересекается по целям с несколькими другими, но не всеми.<br>
Это характерно для препаратов, имеющих ограниченный, но не уникальный диапазон действия.<br>

Низкое значение (avg_target_overlap < 0.5):<br>
Лекарство имеет мало общих целей с другими.<br>
Это может указывать на узконаправленное или уникальное действие препарата.<br>

Принцип работы столбца avg_mechanism_similarity<br>
Этот столбец вычисляет среднее значение коэффициента Жаккара между текстом механизма действия текущего лекарства и текстами механизмов действия всех других лекарств в наборе данных.<br>

Коэффициент Жаккара измеряет степень схожести двух множеств.<br>
Как анализировать данный столбец:<br>

Высокое значение (avg_mechanism_similarity > 0.5):<br>
Лекарство имеет схожие механизмы действия с большинством других лекарств.<br>
Это может указывать на то, что препарат входит в общую терапевтическую группу или имеет стандартный подход к лечению.<br>
Например, противовоспалительные или ингибиторы могут быть схожи.<br>

Среднее значение (0.2 <= avg_mechanism_similarity <= 0.5):<br>
Лекарство частично пересекается с другими по механизму действия.<br>
Это может указывать на узконаправленные, но не уникальные свойства препарата.<br>

Низкое значение (avg_mechanism_similarity < 0.2):<br>
Лекарство обладает уникальным механизмом действия.<br>
Это может быть инновационный препарат или специфический для редкого заболевания.<br>

📌 **Комментарий к коду ниже**:

Первая функция рассчитывает сходство текстов по частоте слов.

Вторая функция рассчитывает среднее количество совпадающих целевых значений между наборами.

In [68]:
from sklearn.feature_extraction.text import CountVectorizer

def calculate_jaccard_similarity_multiclass(text, all_texts):
    vectorizer = CountVectorizer().fit_transform([text] + all_texts)
    vectors = vectorizer.toarray()
    current_vector = vectors[0]
    similarities = [
        np.sum(np.minimum(current_vector, vec)) / np.sum(np.maximum(current_vector, vec))
        for vec in vectors[1:]
    ]
    return np.mean(similarities) if similarities else 0

def calculate_target_overlap(targets, all_targets):
    current_set = set(targets.split(';'))
    overlaps = [
        len(current_set & set(other.split(';'))) for other in all_targets
    ]
    return np.mean(overlaps) if overlaps else 0


📌 **Комментарий к коду ниже**:

---
Код рассчитывает среднее сходство механизмов действия и среднее совпадение целевых значений для каждой строки в DataFrame, исключая строки с отсутствующими данными.


---
**Инициализация столбцов:**

data['avg_mechanism_similarity'] = np.nan и data['avg_target_overlap'] = np.nan: Создает два новых столбца в DataFrame и заполняет их NaN (неопределенными значениями).

**Цикл по строкам DataFrame:**

for index, row in data.iterrows(): - перебирает каждую строку в DataFrame.

**Проверка наличия данных:**

mechanism_missing и targets_missing: Проверяют, отсутствуют ли данные о механизме действия или целевых значениях в текущей строке.

**Подготовка данных для сравнения:**

remaining_mechanisms и remaining_targets: Создают списки механизмов действия и целевых значений из всех остальных строк (исключая текущую).

**Расчет сходства:**

Если механизм действия не отсутствует, рассчитывает среднее сходство Жаккара между механизмом действия в текущей строке и механизмами действия в остальных строках с помощью функции calculate_jaccard_similarity_multiclass.

Если целевые значения не отсутствуют, рассчитывает среднее совпадение целевых значений между текущей строкой и остальными строками с помощью функции calculate_target_overlap.

**Запись результатов:**

Результаты расчетов записываются в соответствующие столбцы (avg_mechanism_similarity и avg_target_overlap) для каждой строки.


In [70]:
import numpy as np
import pandas as pd

data['avg_mechanism_similarity'] = np.nan
data['avg_target_overlap'] = np.nan

for index, row in data.iterrows():
    mechanism_missing = row['Mechanism of Action'] in ["No mechanism found", "Not Available"]
    targets_missing = row['Targets'] == "No targets found"

    remaining_mechanisms = data.loc[data.index != index, 'Mechanism of Action'].tolist()
    remaining_targets = data.loc[data.index != index, 'Targets'].tolist()

    if not mechanism_missing:
        # Проверка и замена NaN перед вызовом функции
        mechanism_of_action = row['Mechanism of Action']
        if isinstance(mechanism_of_action, float) and np.isnan(mechanism_of_action):
            mechanism_of_action = ''  # Заменяем NaN на пустую строку

        # Обработка NaN в remaining_mechanisms
        remaining_mechanisms = [
            '' if isinstance(mech, float) and np.isnan(mech) else mech
            for mech in remaining_mechanisms
        ]

        data.at[index, 'avg_mechanism_similarity'] = calculate_jaccard_similarity_multiclass(
            mechanism_of_action, remaining_mechanisms
        )

    if not targets_missing:
        # Аналогичная обработка для Targets (если необходимо)
        targets = row['Targets']
        if isinstance(targets, float) and np.isnan(targets):
            targets = ''

        remaining_targets = [
            '' if isinstance(targ, float) and np.isnan(targ) else targ
            for targ in remaining_targets
        ]

        data.at[index, 'avg_target_overlap'] = calculate_target_overlap(
            targets, remaining_targets
        )

<ipython-input-68-aab8f564941e>:8: RuntimeWarning: invalid value encountered in scalar divide
  np.sum(np.minimum(current_vector, vec)) / np.sum(np.maximum(current_vector, vec))
<ipython-input-68-aab8f564941e>:8: RuntimeWarning: invalid value encountered in scalar divide
  np.sum(np.minimum(current_vector, vec)) / np.sum(np.maximum(current_vector, vec))
<ipython-input-68-aab8f564941e>:8: RuntimeWarning: invalid value encountered in scalar divide
  np.sum(np.minimum(current_vector, vec)) / np.sum(np.maximum(current_vector, vec))
<ipython-input-68-aab8f564941e>:8: RuntimeWarning: invalid value encountered in scalar divide
  np.sum(np.minimum(current_vector, vec)) / np.sum(np.maximum(current_vector, vec))
<ipython-input-68-aab8f564941e>:8: RuntimeWarning: invalid value encountered in scalar divide
  np.sum(np.minimum(current_vector, vec)) / np.sum(np.maximum(current_vector, vec))
<ipython-input-68-aab8f564941e>:8: RuntimeWarning: invalid value encountered in scalar divide
  np.sum(np.mini

In [71]:
data

,ATC_info,ATC_hierarchy,used_in_diabetes,is_terminal_category,number_of_children,Mechanism of Action,Targets,Indication,ATC Code,Drug Name,...,num_targets,mechanism_complexity,indication_category,num_dfg,num_ingridients,missing_mechanism,missing_targets,missing_indication,avg_mechanism_similarity,avg_target_overlap
ATC,,,,,,,,,,,,,,,,,,,,,
A07AA10,Colistin,"['A07AA10', 'A07AA', 'A07A', 'A07', 'A']",True,1,1,Colistin is a surface active agent which penet...,"ADihydropteroate synthase (binder, Escherichia...",For the treatment of acute or chronic infectio...,A07AA,Colistin,...,3,54,chronic,4,1,False,False,False,0.033114,0.000000
A08AA10,Sibutramine,"['A08AA10', 'A08AA', 'A08A', 'A08', 'A']",True,1,1,Sibutramine produces its therapeutic effects b...,ASodium-dependent dopamine transporter (inhibi...,For the treatment of obesity.,A08AA,Sibutramine,...,4,93,obesity,2,1,False,False,False,0.044317,0.001129
A10AB01,Insulin (human),"['A10AB01', 'A10AB', 'A10A', 'A10', 'A']",True,1,1,The primary activity of insulin is the regulat...,"AInsulin receptor (agonist, Humans); UInsulin-...",Human insulin is indicated to improve glycemic...,A10AB,Insulin (human),...,7,160,other,2,3,False,False,False,0.104252,0.250000
A10AB01,Insulin (human),"['A10AB01', 'A10AB', 'A10A', 'A10', 'A']",True,1,1,The primary activity of insulin is the regulat...,"AInsulin receptor (agonist, Humans); UInsulin-...",Human insulin is indicated to improve glycemic...,A10AB,Insulin (pork),...,7,160,other,1,3,False,False,False,0.104252,0.250000
A10AB01,Insulin (human),"['A10AB01', 'A10AB', 'A10A', 'A10', 'A']",True,1,1,The primary activity of insulin is the regulat...,"AInsulin receptor (agonist, Humans); UInsulin-...",Human insulin is indicated to improve glycemic...,A10AB,Insulin (human),...,7,160,other,2,3,False,False,False,0.104252,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
N05BA10,Ketazolam,"['N05BA10', 'N05BA', 'N05B', 'N05', 'N']",True,1,1,Benzodiazepines share a similar chemical struc...,AGABA(A) Receptor (positive allosteric modulat...,Ketazolam could be used for the treatment of a...,N05BA,Ketazolam,...,3,127,other,0,1,False,False,False,0.051219,0.000000
N05CA10,Cyclobarbital,"['N05CA10', 'N05CA', 'N05C', 'N05', 'N']",True,1,1,Not Available,No targets found,Not Available,N05CA,Cyclobarbital,...,1,2,other,0,1,True,True,True,NaN,NaN
N06AA10,Nortriptyline,"['N06AA10', 'N06AA', 'N06A', 'N06', 'N']",True,1,1,Though prescribing information does not identi...,ASodium-dependent noradrenaline transporter (i...,Nortriptyline is indicated for the relief of t...,N06AA,Nortriptyline,...,16,91,chronic,0,1,False,False,False,0.037143,0.002257


In [72]:
data.to_csv('data_for_eda.csv')